# 卷积神经网络---使用gluon
现在我们使用gluon来实现 -- 上一章的卷积神经网络 

# 定义模型 
下面是LeNet在gluon里的实现，注意到我们不再需要实现去计算每层的输入大小，尤其是接在卷积后面的那个全连接层。 

In [1]:
from mxnet import gluon 
net=gluon.nn.Sequential() 

with net.name_scope():
    net.add(gluon.nn.Conv2D(channels=20,kernel_size=5,activation='relu'))
    net.add(gluon.nn.MaxPool2D(pool_size=2,strides=2))
    net.add(gluon.nn.Conv2D(channels=50,kernel_size=3,activation='relu'))
    net.add(gluon.nn.MaxPool2D(pool_size=2,strides=2))
    net.add(gluon.nn.Flatten())
    net.add(gluon.nn.Dense(128,activation='relu'))
    net.add(gluon.nn.Dense(10))

然后我们尝试将模型权重初始化在GPU上 

In [2]:
import sys 
import d2lzh
sys.path.append('..')  

ctx=d2lzh.utils.try_gpu()
net.initialize(ctx=ctx) 
print("initialize weight on",ctx) 


initialize weight on gpu(0)


# 获取数据
跟之前没什么两样 。

In [3]:
from mxnet import autograd as autograd 
from mxnet import nd
import d2lzh_my
import time
batch_size=256
train_data,test_data=d2lzh.utils.load_data_fashion_mnist(batch_size) 

softmax_cross_entropy=gluon.loss.SoftmaxCrossEntropyLoss()
trainer=gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':0.5})
begin = time.time() 
for epoch in range(5):
    train_loss=0. 
    train_acc=0. 
    for data,label in train_data:
        label=label.as_in_context(ctx) 
        with autograd.record():
            output=net(data.as_in_context(ctx)) 
            loss=softmax_cross_entropy(output,label) 
        loss.backward()
        trainer.step(batch_size) 
        train_loss+=nd.mean(loss).asscalar()
#         train_acc+=d2lzh_my.utils.accuracy(output,label.astype("float32"))
        train_acc+=d2lzh_my.utils.accuracy(output,label.astype("float32"))
    test_acc=d2lzh_my.utils.evaluate_accuracy(test_data,net,ctx) 
    print("Epoch %d . train loss  :%f ,train acc %f,Test acc%f "% 
          (epoch,train_loss/len(train_data),train_acc.asscalar()/len(train_data),test_acc)
          ) 
end=time.time()
print(end-begin)

Epoch 0 . train loss  :1.428824 ,train acc 0.474645,Test acc0.727500 
Epoch 1 . train loss  :0.539647 ,train acc 0.793085,Test acc0.839100 
Epoch 2 . train loss  :0.436536 ,train acc 0.835805,Test acc0.857600 
Epoch 3 . train loss  :0.379627 ,train acc 0.858261,Test acc0.870100 
Epoch 4 . train loss  :0.344304 ,train acc 0.869819,Test acc0.875200 
13.472557783126831
